# Инициализация данных

In [1]:
import os
import pandas as pd

from datetime import datetime
from catboost import CatBoostClassifier
from loguru import logger

from dotenv import load_dotenv

import hashlib
from pydantic import BaseModel
from typing import List

load_dotenv()

True

In [2]:
salt = 'get_exp_group'
num_groups = 2

In [3]:
# Разбиение пользователей на группы
def get_exp_group(user_id: int) -> str:
    exp_group = int(hashlib.md5((str(user_id) + salt).encode()).hexdigest(), 16) % num_groups
    
    if exp_group == 0:
        return 'control'
    else:
        return 'test'

In [4]:
get_exp_group(200)

'control'

In [5]:
class PostGet(BaseModel):
    id: int
    text: str
    topic: str
    
    class Config:
        orm_mode = True

class Response(BaseModel):
    exp_group: str
    recommendations: List[PostGet]

/Users/dmitry/anaconda3/envs/ml/lib/python3.11/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [6]:
# Путь модели
def get_model_path(model_name: str) -> str:
    if os.environ.get("IS_LMS") == "1":
        if model_name == 'model_control':
            MODEL_PATH = '/workdir/user_input/model_control'
        elif model_name == 'model_test':
            MODEL_PATH = '/workdir/user_input/model_test'
        else:
            raise ValueError('unknown name')
    
    else:
        if model_name == 'model_control':
            MODEL_PATH = '/Users/dmitry/Documents/code/Start_ML/module_2/final_project/model/catboost_ml_1'
        elif model_name == 'model_test':
            MODEL_PATH = '/Users/dmitry/Documents/code/Start_ML/module_2/final_project/model/catboost_dl_3'
        else:
            raise ValueError('unknown name')
    return MODEL_PATH


# Загрузка модели
def load_models(model_name: str):
    model_path = get_model_path(model_name)

    from_file = CatBoostClassifier()
    model = from_file.load_model(model_path, format='cbm')
    return model

# Основная функция загрузки признаков из БД
def load_features():
    # Признаки по постам (созданные) TF-IDF
    logger.info("loading tf-idf post feachures")
    post_feachures_control = pd.read_sql(
        """
        SELECT *
        FROM d_trubitsin_post_feachures_base 
        """,
        con=os.environ["POSTGRES_CONN"]
    )

    # Признаки по постам (созданные) DL
    logger.info("loading dl post feachures")
    post_feachures_test = pd.read_sql(
        """
        SELECT *
        FROM d_trubitsin_post_feachures_dl 
        """,
        con=os.environ["POSTGRES_CONN"]
    )

    # Признаки по пользователям
    logger.info("loading user feachures")
    user_feachures = pd.read_sql(
        """
        SELECT *
        FROM public.user_data
        """,
        con=os.environ["POSTGRES_CONN"]
    )
    return [post_feachures_control, post_feachures_test, user_feachures]

In [7]:
logger.info("loading model")
model_control = load_models("model_control")
model_test = load_models("model_test")

2024-04-17 21:12:21.147 | INFO     | __main__:<module>:1 - loading model


In [10]:
logger.info("loading feachures")
feachures = load_features()
logger.info("service is up and running")

2024-04-17 21:12:29.236 | INFO     | __main__:<module>:1 - loading feachures
2024-04-17 21:12:29.237 | INFO     | __main__:load_features:32 - loading tf-idf post feachures
2024-04-17 21:12:33.385 | INFO     | __main__:load_features:42 - loading dl post feachures
2024-04-17 21:12:37.097 | INFO     | __main__:load_features:52 - loading user feachures
2024-04-17 21:12:41.350 | INFO     | __main__:<module>:3 - service is up and running


# Работа endpoint

In [11]:
id = 2001
time = datetime(year=2021, month=11, day=3, hour=14)
limit = 5

In [12]:
def get_recommendations(id, time, model, post_feachures, user_feachures):
   
    # Признаки пользователя
    logger.info(f"user_id: {id}")
    logger.info("reading feachures")
    user_feach = user_feachures.loc[user_feachures["user_id"] == id].copy()
    user_feach.drop(['user_id', 'os', 'source'], axis=1, inplace=True)

    # Признаки постов
    logger.info("dropping columns")
    post_feach = post_feachures.drop(["index", "text"], axis=1)
    content = post_feachures[["post_id", "text", "topic"]]

    # Объединение признаков
    logger.info("zipping everything")
    add_user_feachures = dict(
        zip(user_feach.columns, user_feach.values[0]))
    request_data = post_feach.assign(**add_user_feachures)
    request_data = request_data.set_index('post_id')

    # Добавление даты
    logger.info("adding time info")
    request_data['weekday'] = time.weekday()
    request_data['hour'] = time.hour

    # Предсказание вероятности для 1 класса (вероятнее всего понравится)
    logger.info("predicting")
    probabilities = model.predict_proba(request_data)[:, 1]
    request_data['prediction'] = probabilities

    # Получение топ-5 индексов вероятностей
    recommended_posts = request_data.sort_values("prediction")[-limit:].index

    return [PostGet(**{
        'id': i,
        'text': content[content['post_id'] == i]['text'].values[0],
        'topic': content[content['post_id'] == i]['topic'].values[0]
        }) for i in recommended_posts]


In [15]:
def recommended_posts(id, time):
    # Отнесение пользователя к группе
    logger.info('getting exp group')
    exp_group = get_exp_group(id)
    logger.info(f"User '{id}' assigned to '{exp_group}'")

    if exp_group == 'control':
        recommendations = get_recommendations(id, time, model_control, feachures[0], feachures[2])
    elif exp_group == 'test':
        recommendations = get_recommendations(id, time, model_test, feachures[1], feachures[2])
    else:
        raise ValueError('unknown group')
    
    return Response(exp_group=exp_group, recommendations=recommendations).model_dump_json()

In [16]:
recommended_posts(id, time)

2024-04-17 21:13:28.150 | INFO     | __main__:recommended_posts:3 - getting exp group
2024-04-17 21:13:28.151 | INFO     | __main__:recommended_posts:5 - User '2001' assigned to 'control'
2024-04-17 21:13:28.153 | INFO     | __main__:get_recommendations:4 - user_id: 2001
2024-04-17 21:13:28.154 | INFO     | __main__:get_recommendations:5 - reading feachures
2024-04-17 21:13:28.156 | INFO     | __main__:get_recommendations:10 - dropping columns
2024-04-17 21:13:28.159 | INFO     | __main__:get_recommendations:15 - zipping everything
2024-04-17 21:13:28.165 | INFO     | __main__:get_recommendations:22 - adding time info
2024-04-17 21:13:28.178 | INFO     | __main__:get_recommendations:26 - ['topic', 'TextCluster', 'DistanceTo1thCluster', 'DistanceTo2thCluster', 'DistanceTo3thCluster', 'DistanceTo4thCluster', 'DistanceTo5thCluster', 'DistanceTo6thCluster', 'DistanceTo7thCluster', 'DistanceTo8thCluster', 'DistanceTo9thCluster', 'DistanceTo10thCluster', 'gender', 'age', 'country', 'city', '

'{"exp_group":"control","recommendations":[{"id":6713,"text":"The Seven-Ups is a good and engrossing film. Its packed with credible performances by Scheider, LaBianco and an effective scary performance by Richard Lynch - although most of the characters are card-board cut-out tough guys. Character development does not evolve at all on the screen. The only thing we know is the good guys are the good guys and the bad guys are bad. Deviating from the crime story norm, The Seven-Ups manage to throw Scheider and crew into the middle of a building plot in a unique writing twist. Onsite locations of New York City and an excellent choreographed car chase highlight the film. The only downside of the film is the slightly confusing plot line in the beginning. They give the viewer little evidence that the men being kidnapped are mob related (until later in the film). Had someone blindly started watching the film may be slightly confused on the story. Otherwise, The Seven-Ups is a gritty, testostero

In [122]:
def recommended_posts(
        id: int,
        time: datetime,
        limit: int = 5):

    # Признаки пользователя
    logger.info(f"user_id: {id}")
    logger.info("reading feachures")
    user_feachures = feachures[2].loc[feachures[2]["user_id"] == id].copy()
    user_feachures.drop('user_id', axis=1, inplace=True)

    # Признаки постов
    logger.info("dropping columns")
    post_feachures = feachures[1].drop(["index", "text"], axis=1)
    content = feachures[1][["post_id", "text", "topic"]]

    # Объединение признаков
    logger.info("zipping everything")
    request_data = pd.merge(
        user_feachures, post_feachures, how='cross')
    request_data = request_data.set_index("post_id")

    # Добавление даты
    logger.info("adding time info")
    request_data['weekday'] = time.weekday()
    request_data['hour'] = time.hour


    # Предсказание вероятности для 1 класса (вероятнее всего понравится)
    logger.info("predicting")
    probabilities = model.predict_proba(request_data)[:, 1]
    request_data['prediction'] = probabilities

    # Убираем посты, где пользователь уже поставил "like"
    logger.info("deleting liked posts")
    liked_posts = feachures[0]
    liked_posts = liked_posts[liked_posts["user_id"] == id]["post_id"].values
    filtered_ = request_data[~request_data.index.isin(liked_posts)]

    # Получение топ-5 индексов вероятностей
    recommended_posts = filtered_.sort_values("prediction")[-limit:].index
    response = content.iloc[recommended_posts].copy()

    # Форматирование выдачи согласно шаблону
    response.rename(columns={'post_id': 'id'}, inplace=True)
    result_list = response.to_dict(orient='records')

    return result_list

In [123]:
recommended_posts(id, time)

2024-04-16 18:09:42.033 | INFO     | __main__:recommended_posts:7 - user_id: 200
2024-04-16 18:09:42.034 | INFO     | __main__:recommended_posts:8 - reading feachures


IndexError: list index out of range